1- from_multiple_xml_to_csv

In [5]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import pandas as pd
from lxml import etree
from IPython.display import display

def read_xml(file_path):
    parser = etree.XMLParser(recover=True)

    with open(file_path, encoding="utf-8") as file:
        data = file.read()

    raw = etree.fromstring(data, parser=parser)
    return raw


def events_to_df(eventlist):
    df_list = []
    tag = '{http://schemas.microsoft.com/win/2004/08/events/event}'
    for idx, event in enumerate(eventlist):
        edict = {}
        for element in event.iterdescendants():
            if any(x in element.tag for x in ['TimeCreated', 'Execution', 'Security']):
                for item in element.items():
                    edict[item[0]] = item[1]
            elif any(x in element.tag for x in ['Provider', 'System', 'Correlation']):
                pass
            elif 'Data' in element.tag:
                for item in element.items():
                    edict[item[1]] = element.text
            else:
                edict[element.tag.replace(tag, '')] = element.text

        edict['raw'] = etree.tostring(event, pretty_print=True).decode()

        edf = pd.DataFrame(edict, index=[idx])
        df_list.append(edf)

    return pd.concat(df_list, sort=True)

def process_xml_folder(folder_path, output_folder='csv'):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.endswith(".xml"):
            xml_file_path = os.path.join(folder_path, filename)
            output_csv_path = os.path.join(output_folder, filename.replace(".xml", ".csv"))

            print(f"Processing {xml_file_path} and saving to {output_csv_path}")
            xml_data = read_xml(xml_file_path)
            events = [element for element in xml_data.iter('{http://schemas.microsoft.com/win/2004/08/events/event}Event')]
            log_df = events_to_df(events)
            log_df.to_csv(output_csv_path, index=False)

# Provide the folder path containing XML files
xml_folder_path = "xml"

# Call the function to process the XML folder
process_xml_folder(xml_folder_path)


Processing xml\apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.xml and saving to csv\apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.csv
Processing xml\CA_Mimikatz_Memssp_Default_Logs_Sysmon_11.xml and saving to csv\CA_Mimikatz_Memssp_Default_Logs_Sysmon_11.csv
Processing xml\CA_sysmon_hashdump_cmd_meterpreter.xml and saving to csv\CA_sysmon_hashdump_cmd_meterpreter.csv
Processing xml\CA_teamviewer-dumper_sysmon_10.xml and saving to csv\CA_teamviewer-dumper_sysmon_10.csv
Processing xml\CVE-2020-0796_SMBV3Ghost_LocalPrivEsc_Sysmon_3_1_10.xml and saving to csv\CVE-2020-0796_SMBV3Ghost_LocalPrivEsc_Sysmon_3_1_10.csv
Processing xml\DE_BYOV_Zam64_CA_Memdump_sysmon_7_10.xml and saving to csv\DE_BYOV_Zam64_CA_Memdump_sysmon_7_10.csv
Processing xml\de_portforward_netsh_rdp_sysmon_13_1.xml and saving to csv\de_portforward_netsh_rdp_sysmon_13_1.csv
Processing xml\DE_Powershell_CLM_Disabled_Sysmon_12.xml and saving to csv\DE_Powershell_CLM_Disabled_Sy

2- feature selected

In [8]:
import os
import pandas as pd

# Input and output folders
input_folder = r"csv"
output_folder = r"selected_features"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Features to select
selected_features = [
    "CommandLine", "Company", "CurrentDirectory", "DestinationIsIpv6", "DestinationPort",
    "DestinationPortName", "Details", "EventID", "EventType", "FileVersion", "Initiated",
    "IntegrityLevel", "Level", "Message", "ParentCommandLine", "ParentUser", "Product",
    "Protocol", "QueryName", "QueryResults", "QueryStatus", "RuleName", "SourceImage",
    "SourceIsIpv6", "SourcePort", "SourcePortName", "StartFunction", "StartModule",
    "TargetFilename", "TargetImage", "TargetUser", "Task"
]

# Iterate over each CSV file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        # Read the CSV file
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        # Initialize a DataFrame with zeros for selected features
        selected_df = pd.DataFrame(0, index=df.index, columns=selected_features)

        # Fill in values for existing features
        for feature in selected_features:
            if feature in df.columns:
                selected_df[feature] = df[feature]

        # Print debugging information
        print(f"Processed: {filename}")
        print(f"Original CSV columns: {df.columns}")
        print(f"Selected features: {selected_features}")
        print(f"Selected DataFrame columns: {selected_df.columns}")
        print(f"Selected DataFrame shape: {selected_df.shape}")

        # Write the selected features to a new CSV file in the output folder
        output_path = os.path.join(output_folder, filename)
        selected_df.to_csv(output_path, index=False)

        print(f"Output CSV: {output_path}")


Processed: apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.csv
Original CSV columns: Index(['Channel', 'CommandLine', 'Company', 'Computer', 'CurrentDirectory',
       'Description', 'Details', 'EventID', 'EventRecordID', 'EventType',
       'FileVersion', 'Hashes', 'Image', 'ImageLoaded', 'IntegrityLevel',
       'Keywords', 'Level', 'LogonGuid', 'LogonId', 'NewThreadId', 'Opcode',
       'ParentCommandLine', 'ParentImage', 'ParentProcessGuid',
       'ParentProcessId', 'ProcessGuid', 'ProcessID', 'ProcessId', 'Product',
       'RuleName', 'Signature', 'SignatureStatus', 'Signed', 'SourceImage',
       'SourceProcessGuid', 'SourceProcessId', 'StartAddress', 'StartFunction',
       'StartModule', 'SystemTime', 'TargetImage', 'TargetObject',
       'TargetProcessGuid', 'TargetProcessId', 'Task', 'TerminalSessionId',
       'ThreadID', 'User', 'UserID', 'UtcTime', 'Version', 'raw'],
      dtype='object')
Selected features: ['CommandLine', 'Company', 'CurrentDirecto

3- replce null with 0

In [9]:
import pandas as pd
import os

# Set the path to the directory containing your CSV files
input_csv_directory = 'selected_features'

# Set the path to the directory where you want to save the updated CSV files
output_csv_directory = 'without null'

# Create the output directory if it doesn't exist
os.makedirs(output_csv_directory, exist_ok=True)

# Iterate over all CSV files in the input directory
for filename in os.listdir(input_csv_directory):
    if filename.endswith('.csv'):
        input_file_path = os.path.join(input_csv_directory, filename)

        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(input_file_path)

        # Fill null values with 0
        df.fillna(0, inplace=True)

        # Set the path for the output file in the new directory
        output_file_path = os.path.join(output_csv_directory, filename)

        # Save the updated DataFrame to the output CSV file
        df.to_csv(output_file_path, index=False)

        print(f'Processed: {filename}')

print('All files processed and saved to the output directory.')


Processed: apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.csv
Processed: CA_Mimikatz_Memssp_Default_Logs_Sysmon_11.csv
Processed: CA_sysmon_hashdump_cmd_meterpreter.csv
Processed: CA_teamviewer-dumper_sysmon_10.csv
Processed: CVE-2020-0796_SMBV3Ghost_LocalPrivEsc_Sysmon_3_1_10.csv
Processed: DE_BYOV_Zam64_CA_Memdump_sysmon_7_10.csv
Processed: de_portforward_netsh_rdp_sysmon_13_1.csv
Processed: DE_Powershell_CLM_Disabled_Sysmon_12.csv
Processed: de_powershell_execpolicy_changed_sysmon_13.csv
Processed: DE_ProcessHerpaderping_Sysmon_11_10_1_7.csv
Processed: de_PsScriptBlockLogging_disabled_sysmon12_13.csv
Processed: DE_remote_eventlog_svc_crash_byt3bl33d3r_sysmon_17_1_3.csv
Processed: DE_renamed_psexec_service_sysmon_17_18.csv
Processed: DE_sysmon-3-rdp-tun.csv
Processed: de_sysmon_13_VBA_Security_AccessVBOM.csv
Processed: DE_UAC_Disabled_Sysmon_12_13.csv
Processed: de_unmanagedpowershell_psinject_sysmon_7_8_10.csv
Processed: discovery_enum_shares_target_sysmon_3_

4- fixed length

In [10]:
import pandas as pd
import os

# Define the column lengths
column_lengths = {
    "CommandLine": 350,
    "Company": 36,
    "CurrentDirectory": 94,
    "DestinationIsIpv6": 5,
    "DestinationPort": 4,
    "DestinationPortName": 13,
    "Details": 269,
    "EventID": 2,
    "EventType": 12,
    "FileVersion": 59,
    "Initiated": 4,
    "IntegrityLevel": 6,
    "Level": 12,
    "Message": 500,
    "ParentCommandLine": 500,
    "ParentUser": 29,
    "Product": 39,
    "Protocol": 3,
    "QueryName": 254,
    "QueryResults": 400,
    "QueryStatus": 5,
    "RuleName": 35,
    "SourceImage": 58,
    "SourceIsIpv6": 5,
    "SourcePort": 5,
    "SourcePortName": 1,
    "StartFunction": 12,
    "StartModule": 35,
    "TargetFilename": 204,
    "TargetImage": 48,
    "TargetUser": 20,
    "Task": 55,
}

# Specify the directory containing the CSV files
input_directory = "without null"

# Specify the output directory for the modified CSV files
output_directory = "fixed length"

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over the CSV files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_filepath = os.path.join(input_directory, filename)
        output_filepath = os.path.join(output_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(input_filepath)

        # Replace null values with zeros
        df = df.fillna(0)

        # Iterate over all columns and update values to fixed length
        for column, length in column_lengths.items():
            if column in df.columns:
                df[column] = df[column].apply(lambda x: str(x).zfill(length))

        # Write the modified DataFrame to a new CSV file
        df.to_csv(output_filepath, index=False)

        print(f"Processed: {filename}")

print("Processing complete.")


Processed: apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.csv
Processed: CA_Mimikatz_Memssp_Default_Logs_Sysmon_11.csv
Processed: CA_sysmon_hashdump_cmd_meterpreter.csv
Processed: CA_teamviewer-dumper_sysmon_10.csv
Processed: CVE-2020-0796_SMBV3Ghost_LocalPrivEsc_Sysmon_3_1_10.csv
Processed: DE_BYOV_Zam64_CA_Memdump_sysmon_7_10.csv
Processed: de_portforward_netsh_rdp_sysmon_13_1.csv
Processed: DE_Powershell_CLM_Disabled_Sysmon_12.csv
Processed: de_powershell_execpolicy_changed_sysmon_13.csv
Processed: DE_ProcessHerpaderping_Sysmon_11_10_1_7.csv
Processed: de_PsScriptBlockLogging_disabled_sysmon12_13.csv
Processed: DE_remote_eventlog_svc_crash_byt3bl33d3r_sysmon_17_1_3.csv
Processed: DE_renamed_psexec_service_sysmon_17_18.csv
Processed: DE_sysmon-3-rdp-tun.csv
Processed: de_sysmon_13_VBA_Security_AccessVBOM.csv
Processed: DE_UAC_Disabled_Sysmon_12_13.csv
Processed: de_unmanagedpowershell_psinject_sysmon_7_8_10.csv
Processed: discovery_enum_shares_target_sysmon_3_

5- combine all columns

In [11]:
import pandas as pd
import os

# Specify the directory containing the CSV files
input_directory = "fixed length"

# Specify the output directory for the combined CSV files
output_directory = "combined"

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over the CSV files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_filepath = os.path.join(input_directory, filename)
        output_filepath = os.path.join(output_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(input_filepath)

        # Combine all column values into a single feature vector
        df['FeatureVector'] = df.apply(lambda row: ''.join(map(str, row)), axis=1)

        # Save the modified DataFrame to a new CSV file
        df[['FeatureVector']].to_csv(output_filepath, index=False)

        print(f"Processed: {filename}")

print("Processing complete.")


Processed: apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.csv
Processed: CA_Mimikatz_Memssp_Default_Logs_Sysmon_11.csv
Processed: CA_sysmon_hashdump_cmd_meterpreter.csv
Processed: CA_teamviewer-dumper_sysmon_10.csv
Processed: CVE-2020-0796_SMBV3Ghost_LocalPrivEsc_Sysmon_3_1_10.csv
Processed: DE_BYOV_Zam64_CA_Memdump_sysmon_7_10.csv
Processed: de_portforward_netsh_rdp_sysmon_13_1.csv
Processed: DE_Powershell_CLM_Disabled_Sysmon_12.csv
Processed: de_powershell_execpolicy_changed_sysmon_13.csv
Processed: DE_ProcessHerpaderping_Sysmon_11_10_1_7.csv
Processed: de_PsScriptBlockLogging_disabled_sysmon12_13.csv
Processed: DE_remote_eventlog_svc_crash_byt3bl33d3r_sysmon_17_1_3.csv
Processed: DE_renamed_psexec_service_sysmon_17_18.csv
Processed: DE_sysmon-3-rdp-tun.csv
Processed: de_sysmon_13_VBA_Security_AccessVBOM.csv
Processed: DE_UAC_Disabled_Sysmon_12_13.csv
Processed: de_unmanagedpowershell_psinject_sysmon_7_8_10.csv
Processed: discovery_enum_shares_target_sysmon_3_

6- transpose all files

In [12]:
import pandas as pd
import os

# Specify the directory containing the CSV files
input_directory = "combined"

# Specify the output directory for the transposed CSV files
output_directory = "transpos"

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over the CSV files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_filepath = os.path.join(input_directory, filename)
        output_filepath = os.path.join(output_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(input_filepath)

        # Transpose the DataFrame (swap rows and columns)
        df_transposed = df.transpose()

        # Save the transposed DataFrame to a new CSV file
        df_transposed.to_csv(output_filepath, header=False)

        print(f"Processed: {filename}")

print("Processing complete.")


Processed: apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.csv
Processed: CA_Mimikatz_Memssp_Default_Logs_Sysmon_11.csv
Processed: CA_sysmon_hashdump_cmd_meterpreter.csv
Processed: CA_teamviewer-dumper_sysmon_10.csv
Processed: CVE-2020-0796_SMBV3Ghost_LocalPrivEsc_Sysmon_3_1_10.csv
Processed: DE_BYOV_Zam64_CA_Memdump_sysmon_7_10.csv
Processed: de_portforward_netsh_rdp_sysmon_13_1.csv
Processed: DE_Powershell_CLM_Disabled_Sysmon_12.csv
Processed: de_powershell_execpolicy_changed_sysmon_13.csv
Processed: DE_ProcessHerpaderping_Sysmon_11_10_1_7.csv
Processed: de_PsScriptBlockLogging_disabled_sysmon12_13.csv
Processed: DE_remote_eventlog_svc_crash_byt3bl33d3r_sysmon_17_1_3.csv
Processed: DE_renamed_psexec_service_sysmon_17_18.csv
Processed: DE_sysmon-3-rdp-tun.csv
Processed: de_sysmon_13_VBA_Security_AccessVBOM.csv
Processed: DE_UAC_Disabled_Sysmon_12_13.csv
Processed: de_unmanagedpowershell_psinject_sysmon_7_8_10.csv
Processed: discovery_enum_shares_target_sysmon_3_

7- remove the first cell

In [13]:
import pandas as pd
import os

# Specify the directory containing the CSV files
input_directory = "transpos"

# Specify the output directory for the modified CSV files
output_directory = "remove"

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over the CSV files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_filepath = os.path.join(input_directory, filename)
        output_filepath = os.path.join(output_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(input_filepath)

        # Check if "FeatureVector" is present in the columns and remove it
        if "FeatureVector" in df.columns:
            df = df.drop(columns=["FeatureVector"])

        # Write the modified DataFrame to a new CSV file
        df.to_csv(output_filepath, index=False)

        print(f"Processed: {filename}")

print("Processing complete. Output saved to:", output_directory)


Processed: apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.csv
Processed: CA_Mimikatz_Memssp_Default_Logs_Sysmon_11.csv
Processed: CA_sysmon_hashdump_cmd_meterpreter.csv
Processed: CA_teamviewer-dumper_sysmon_10.csv
Processed: CVE-2020-0796_SMBV3Ghost_LocalPrivEsc_Sysmon_3_1_10.csv
Processed: DE_BYOV_Zam64_CA_Memdump_sysmon_7_10.csv
Processed: de_portforward_netsh_rdp_sysmon_13_1.csv
Processed: DE_Powershell_CLM_Disabled_Sysmon_12.csv
Processed: de_powershell_execpolicy_changed_sysmon_13.csv
Processed: DE_ProcessHerpaderping_Sysmon_11_10_1_7.csv
Processed: de_PsScriptBlockLogging_disabled_sysmon12_13.csv
Processed: DE_remote_eventlog_svc_crash_byt3bl33d3r_sysmon_17_1_3.csv
Processed: DE_renamed_psexec_service_sysmon_17_18.csv
Processed: DE_sysmon-3-rdp-tun.csv
Processed: de_sysmon_13_VBA_Security_AccessVBOM.csv
Processed: DE_UAC_Disabled_Sysmon_12_13.csv
Processed: de_unmanagedpowershell_psinject_sysmon_7_8_10.csv
Processed: discovery_enum_shares_target_sysmon_3_

8- combine all files into one csv file

In [14]:
import pandas as pd
import os

# Specify the directory containing the CSV files
input_directory = "remove"

# Specify the output file for the combined CSV rows
output_filepath = "malicious.csv"

# Initialize an empty list to store DataFrames from each CSV file
dfs = []

# Iterate over the CSV files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_filepath = os.path.join(input_directory, filename)

        # Read the CSV file into a DataFrame and append to the list
        df = pd.read_csv(input_filepath, header=None)  # Assuming no header in each file
        dfs.append(df)

        print(f"Processed: {filename}")

# Concatenate all DataFrames vertically
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(output_filepath, index=False, header=False)  # No header in the output

print("Combining complete. Output saved to:", output_filepath)


Processed: apt10_jjs_sideloading_prochollowing_persist_as_service_sysmon_1_7_8_13.csv
Processed: CA_Mimikatz_Memssp_Default_Logs_Sysmon_11.csv
Processed: CA_sysmon_hashdump_cmd_meterpreter.csv
Processed: CA_teamviewer-dumper_sysmon_10.csv
Processed: CVE-2020-0796_SMBV3Ghost_LocalPrivEsc_Sysmon_3_1_10.csv
Processed: DE_BYOV_Zam64_CA_Memdump_sysmon_7_10.csv
Processed: de_portforward_netsh_rdp_sysmon_13_1.csv
Processed: DE_Powershell_CLM_Disabled_Sysmon_12.csv
Processed: de_powershell_execpolicy_changed_sysmon_13.csv
Processed: DE_ProcessHerpaderping_Sysmon_11_10_1_7.csv
Processed: de_PsScriptBlockLogging_disabled_sysmon12_13.csv
Processed: DE_remote_eventlog_svc_crash_byt3bl33d3r_sysmon_17_1_3.csv
Processed: DE_renamed_psexec_service_sysmon_17_18.csv
Processed: DE_sysmon-3-rdp-tun.csv
Processed: de_sysmon_13_VBA_Security_AccessVBOM.csv
Processed: DE_UAC_Disabled_Sysmon_12_13.csv
Processed: de_unmanagedpowershell_psinject_sysmon_7_8_10.csv
Processed: discovery_enum_shares_target_sysmon_3_